In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql

from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

#scripts
from src import rank_metrics
from src import helpers
from src import table_encoder
from src import metrics

In [2]:
data_path = '../data/csv'

In [3]:
ratings_df, movies_df, encoded_movies_df, tags_df, enoded_tags_df = helpers.load_format_data(data_path)

In [4]:
train, test = helpers.load_format_data_for_model(data_path)

In [5]:
model = helpers.fit_als(train)

In [6]:
allUserRecs, allMovieRecs = helpers.predict_for_all_users(model, test)

RMSE = 1.086579545094526


In [7]:
user_prediction = helpers.predict_for_one_user(model, 5, test, movies_df)
user_prediction

['Little Women (1994)',
 'Aladdin (1992)',
 'Clueless (1995)',
 'Toy Story (1995)',
 'Batman (1989)']

In [8]:
user_liked_movie_ids = helpers.get_user_liked_movie_ids(5, ratings_df)

In [9]:
user_liked_movie_titles = helpers.get_user_liked_movie_titles(5, ratings_df, movies_df)[:5]

In [10]:
rec_movie_ids = helpers.get_ids_from_titles(user_prediction, movies_df)
rec_movie_ids

[261, 588, 39, 1, 592]

In [11]:
metrics.compare_recs_to_user_likes(rec_movie_ids, user_liked_movie_ids, encoded_genres=encoded_movies_df)

,movie ids,similarity
0,261,0.191866
1,588,0.288236
2,39,0.207497
3,1,0.313626
4,592,0.256397


In [12]:
metrics.compare_movie(232, 984, encoded_movies_df)

0.8165